In [2]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

In [23]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=estimates;'
                    'Trusted_Connection=yes;')

with open(r'sql_queries\dof_query.sql', 'r') as sql_file:
    sql_query = sql_file.read()

dof_data =  pd.read_sql_query(sql_query, conn)
dof_data = dof_data.set_index(['region', 'yr_id'])
dof_data

Total Population  Household Population  Total GQ Population  \
region    yr_id                                                                
San Diego 2020            3331279               3218869               112410   
          2021            3288503               3185504               102999   
          2022            3287306               3170895               116411   

                 Total Households  Single Family - Detached  \
region    yr_id                                               
San Diego 2020            1226879                    622861   
          2021            1234243                    623705   
          2022            1244854                    626055   

                 Single Family - Multiple Unit  two_to_four  five_plus  \
region    yr_id                                                          
San Diego 2020                          451206        87814     363392   
          2021                          457367        89145     368222   
          2022                          464642        89713     374929   

                 Mobile Home  occupied  unoccupied  
region    yr_id                                     
San Diego 2020         46380   1153981       72898  
          2021         46521   1159366       74877  
          2022         46636   1169378       75476

In [33]:
def dof_data_pull():
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=estimates;'
                    'Trusted_Connection=yes;')

    with open(r'sql_queries\dof_query.sql', 'r') as sql_file:
        sql_query = sql_file.read()

    dof_data =  pd.read_sql_query(sql_query, conn)
    dof_data = dof_data.set_index(['region', 'yr_id'])
    return dof_data

In [5]:
# Need Housing and population
def find_and_download_files(url, estimates_version, category):
    path = url.replace("\\", "/")

    files_with_words = []
    for filename in os.listdir(path):
        if 'region' in filename and category in filename and estimates_version in filename:
            file_path = os.path.join(path, filename)
            files_with_words.append(file_path)

    df = pd.read_excel(files_with_words[0])

    return df

In [40]:
def housing_data_pull():
    housing_df = find_and_download_files(url=rf'J:\DataScience\DataQuality\QAQC\Estimates QC Automation\v_series15\individual_files\2022_03', estimates_version='2022_03', category='housing')
    housing_df['unoccupied'] = housing_df['units'] - housing_df['occupied']
    return housing_df

In [14]:
housing_df = find_and_download_files(url=rf'J:\DataScience\DataQuality\QAQC\Estimates QC Automation\v_series15\individual_files\2022_03', estimates_version='2022_03', category='housing')
housing_df['unoccupied'] = housing_df['units'] - housing_df['occupied']
housing_df

,region,yr_id,Mobile Home,Multifamily,Single Family - Detached,Single Family - Multiple Unit,units,occupied,vacancy,unoccupied
0,San Diego,2020,42169,448296,567452,158606,1216523,1144270,72253,72253
1,San Diego,2021,42112,452544,569254,159048,1222958,1148706,74252,74252
2,San Diego,2022,42130,459767,571112,162417,1235426,1160472,74954,74954


In [34]:
def population_data_pull():
    population_df = find_and_download_files(url=rf'J:\DataScience\DataQuality\QAQC\Estimates QC Automation\v_series15\individual_files\2022_03', estimates_version='2022_03', category='population')
    return population_df

In [7]:
population_df = find_and_download_files(url=rf'J:\DataScience\DataQuality\QAQC\Estimates QC Automation\v_series15\individual_files\2022_03', estimates_version='2022_03', category='population')
population_df

,region,yr_id,Group Quarters - College,Group Quarters - Military,Group Quarters - Other,Household Population,Total Population,Total GQ Population
0,San Diego,2020,26601,43188,42621,3218869,3331279,112410
1,San Diego,2021,18015,40360,44624,3185504,3288503,102999
2,San Diego,2022,30992,41841,43578,3170895,3287306,116411


In [35]:
def households_data_pull():
    households_df = find_and_download_files(url=rf'J:\DataScience\DataQuality\QAQC\Estimates QC Automation\v_series15\individual_files\2022_03', estimates_version='2022_03', category='households')
    households_df['two_to_four'] = households_df[2] + households_df[3] + households_df[4]
    households_df['five_plus'] = households_df[5] + households_df[6] + households_df[7]
    return households_df

In [13]:
households_df = find_and_download_files(url=rf'J:\DataScience\DataQuality\QAQC\Estimates QC Automation\v_series15\individual_files\2022_03', estimates_version='2022_03', category='households')
households_df['two_to_four'] = households_df[2] + households_df[3] + households_df[4]
households_df['five_plus'] = households_df[5] + households_df[6] + households_df[7]
households_df

,region,yr_id,1,2,3,4,5,6,7,Total Households,two_to_four,five_plus
0,San Diego,2020,272996,377440,191908,169561,78186,32070,22109,1144270,738909,132365
1,San Diego,2021,275720,377886,193799,168268,78750,31441,22842,1148706,739953,133033
2,San Diego,2022,278544,381757,195784,169992,79556,31763,23076,1160472,747533,134395


In [37]:
def pull_excel_data_and_merge():
    excel_data = households_data_pull().merge(housing_data_pull(), on=['region', 'yr_id']).merge(population_data_pull(), on=['region', 'yr_id'])
    excel_data = excel_data.set_index(['region', 'yr_id'])
    return excel_data

In [22]:
excel_data = housing_df.merge(population_df, on=['region', 'yr_id']).merge(households_df, on=['region', 'yr_id'])
excel_data = excel_data.set_index(['region', 'yr_id'])
excel_data

Mobile Home  Multifamily  Single Family - Detached  \
region    yr_id                                                       
San Diego 2020         42169       448296                    567452   
          2021         42112       452544                    569254   
          2022         42130       459767                    571112   

                 Single Family - Multiple Unit    units  occupied  vacancy  \
region    yr_id                                                              
San Diego 2020                          158606  1216523   1144270    72253   
          2021                          159048  1222958   1148706    74252   
          2022                          162417  1235426   1160472    74954   

                 unoccupied  Group Quarters - College  \
region    yr_id                                         
San Diego 2020        72253                     26601   
          2021        74252                     18015   
          2022        74954                     30992   

                 Group Quarters - Military  ...       1       2       3  \
region    yr_id                             ...                           
San Diego 2020                       43188  ...  272996  377440  191908   
          2021                       40360  ...  275720  377886  193799   
          2022                       41841  ...  278544  381757  195784   

                      4      5      6      7  Total Households  two_to_four  \
region    yr_id                                                               
San Diego 2020   169561  78186  32070  22109           1144270       738909   
          2021   168268  78750  31441  22842           1148706       739953   
          2022   169992  79556  31763  23076           1160472       747533   

                 five_plus  
region    yr_id             
San Diego 2020      132365  
          2021      133033  
          2022      134395  

[3 rows x 24 columns]

In [25]:
def subtract_dataframes(df1, df2):
    # Filter the second data frame to keep only rows with index values in the first data frame
    common_index = pd.merge(df1, df2, left_index=True, right_index=True)
    df1 = df1.loc[common_index.index]
    df2 = df2.loc[common_index.index]
    
    # Find the common columns between the two data frames
    common_cols = list(set(df1.columns) & set(df2.columns))
    common_cols = [x for x in df2.columns if x in common_cols]
    
    # Filter the data frames to keep only the common columns
    df1 = df1[common_cols]
    df2 = df2[common_cols]
    
    # Subtract the second data frame from the first
    result = df1.subtract(df2)
    
    return result

In [26]:
subtract_dataframes(excel_data, dof_data)

Total Population  Household Population  Total GQ Population  \
region    yr_id                                                                
San Diego 2020                  0                     0                    0   
          2021                  0                     0                    0   
          2022                  0                     0                    0   

                 Total Households  Single Family - Detached  \
region    yr_id                                               
San Diego 2020             -82609                    -55409   
          2021             -85537                    -54451   
          2022             -84382                    -54943   

                 Single Family - Multiple Unit  two_to_four  five_plus  \
region    yr_id                                                          
San Diego 2020                         -292600       651095    -231027   
          2021                         -298319       650808    -235189   
          2022                         -302225       657820    -240534   

                 Mobile Home  occupied  unoccupied  
region    yr_id                                     
San Diego 2020         -4211     -9711        -645  
          2021         -4409    -10660        -625  
          2022         -4506     -8906        -522

In [31]:
def percent_diff_dataframes(df1, df2):
    # Filter the second data frame to keep only rows with index values in the first data frame
    common_index = pd.merge(df1, df2, left_index=True, right_index=True)
    df1 = df1.loc[common_index.index]
    df2 = df2.loc[common_index.index]
    
    # Find the common columns between the two data frames
    common_cols = list(set(df1.columns) & set(df2.columns))
    common_cols = [x for x in df2.columns if x in common_cols]
    
    # Filter the data frames to keep only the common columns
    df1 = df1[common_cols]
    df2 = df2[common_cols]
    
    # Subtract the second data frame from the first
    subtraction = df1.subtract(df2)
    division = subtraction.divide(df2)
    result = division*100
    result = result.round(decimals=2)
    return result

In [47]:
def create_output(excel_data, dof_data):
    dof_data = dof_data_pull()
    excel_data = pull_excel_data_and_merge()
    diff_file = subtract_dataframes(excel_data, dof_data)
    pct_diff_file = percent_diff_dataframes(excel_data, dof_data)

    return diff_file, pct_diff_file

In [41]:
diff, pct = create_output()

In [48]:
def write_to_excel(excel_data, dof_data, estimates_version):
    diff, pct = create_output(excel_data, dof_data)
    # Create a Pandas Excel writer using xlsxwriter as the engine
    writer = pd.ExcelWriter(rf'J:\DataScience\DataQuality\QAQC\Estimates QC Automation\v_series15\dof_comparison\{estimates_version}\region_est_minus_dof_QA.xlsx', engine='xlsxwriter')

    # Write each data frame to a different sheet
    excel_data.reset_index().to_excel(writer, sheet_name=f'{estimates_version} Data', index=False)
    dof_data.reset_index().to_excel(writer, sheet_name=f'DOF Data', index=False)
    diff.reset_index().to_excel(writer, sheet_name='Diff', index=False)
    pct.reset_index().to_excel(writer, sheet_name='PCT Diff', index=False)

    # Save the writer
    writer.save()

In [49]:
write_to_excel(excel_data=excel_data, dof_data=dof_data, estimates_version='2022_03')

In [42]:
diff

Total Population  Household Population  Total GQ Population  \
region    yr_id                                                                
San Diego 2020                  0                     0                    0   
          2021                  0                     0                    0   
          2022                  0                     0                    0   

                 Total Households  Single Family - Detached  \
region    yr_id                                               
San Diego 2020             -82609                    -55409   
          2021             -85537                    -54451   
          2022             -84382                    -54943   

                 Single Family - Multiple Unit  two_to_four  five_plus  \
region    yr_id                                                          
San Diego 2020                         -292600       651095    -231027   
          2021                         -298319       650808    -235189   
          2022                         -302225       657820    -240534   

                 Mobile Home  occupied  unoccupied  
region    yr_id                                     
San Diego 2020         -4211     -9711        -645  
          2021         -4409    -10660        -625  
          2022         -4506     -8906        -522

In [43]:
pct

Total Population  Household Population  Total GQ Population  \
region    yr_id                                                                
San Diego 2020                0.0                   0.0                  0.0   
          2021                0.0                   0.0                  0.0   
          2022                0.0                   0.0                  0.0   

                 Total Households  Single Family - Detached  \
region    yr_id                                               
San Diego 2020              -6.73                     -8.90   
          2021              -6.93                     -8.73   
          2022              -6.78                     -8.78   

                 Single Family - Multiple Unit  two_to_four  five_plus  \
region    yr_id                                                          
San Diego 2020                          -64.85       741.45     -63.58   
          2021                          -65.23       730.06     -63.87   
          2022                          -65.04       733.25     -64.15   

                 Mobile Home  occupied  unoccupied  
region    yr_id                                     
San Diego 2020         -9.08     -0.84       -0.88  
          2021         -9.48     -0.92       -0.83  
          2022         -9.66     -0.76       -0.69

In [32]:
percent_diff_dataframes(excel_data, dof_data)

Total Population  Household Population  Total GQ Population  \
region    yr_id                                                                
San Diego 2020                0.0                   0.0                  0.0   
          2021                0.0                   0.0                  0.0   
          2022                0.0                   0.0                  0.0   

                 Total Households  Single Family - Detached  \
region    yr_id                                               
San Diego 2020              -6.73                     -8.90   
          2021              -6.93                     -8.73   
          2022              -6.78                     -8.78   

                 Single Family - Multiple Unit  two_to_four  five_plus  \
region    yr_id                                                          
San Diego 2020                          -64.85       741.45     -63.58   
          2021                          -65.23       730.06     -63.87   
          2022                          -65.04       733.25     -64.15   

                 Mobile Home  occupied  unoccupied  
region    yr_id                                     
San Diego 2020         -9.08     -0.84       -0.88  
          2021         -9.48     -0.92       -0.83  
          2022         -9.66     -0.76       -0.69